In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

input_file = open('/data1/personal/baoyh/emo/model/pred_results/middle/attention_output.npy', 'rb')
attention_output = np.load(input_file)
input_file.close()

input_file = open('/data1/personal/baoyh/emo/model/pred_results/middle/attention_input.npy', 'rb')
attention_input = np.load(input_file)
input_file.close()

attention_output = attention_output[:, :, 1, :]

mean_attention = np.abs(attention_output)
mean_attention = np.mean(mean_attention, axis=2)
mean_attention = np.repeat(mean_attention, 10, axis=1)

mean_attention_input = np.abs(attention_input)
mean_attention_input = np.mean(mean_attention_input, axis=2)
mean_attention_input = np.repeat(mean_attention_input, 10, axis=1)


input_file = open('/data1/personal/baoyh/emo/additional/preprocess_data/middle/train_middle_annotation_MA0079.5_0018_0157_annotation.pkl', 'rb')
test_middle_list = pickle.load(input_file)
input_file.close()
bio_annotation_area = []
no_annotation_area = []
for seq_no in tqdm(range(len(mean_attention))):
    bio_annotation_area.append([])
    no_annotation_area.append([])
    if len(test_middle_list[seq_no]['annotation']) == 0:
        continue # skip if no annotation
    for idx in range(test_middle_list[seq_no]['seq_len']):
        flag = False
        for bio_idx in range(len(test_middle_list[seq_no]['annotation'])):
            if idx >= test_middle_list[seq_no]['annotation'][bio_idx]['start_rel'] and idx <= test_middle_list[seq_no]['annotation'][bio_idx]['end_rel']:
                bio_annotation_area[seq_no].append(mean_attention[seq_no][idx])
                flag = True
                break
        if not flag:
            no_annotation_area[seq_no].append(mean_attention[seq_no][idx])

100%|██████████| 5787/5787 [00:54<00:00, 105.46it/s]


In [6]:
from scipy.stats import mannwhitneyu, ttest_ind
import matplotlib.pyplot as plt
regulation_annotation = pd.read_table('/data1/personal/baoyh/emo/preprocess_data/with_MA0079.5_0018_0157.tsv', sep='\t')
color_list = ['#0000FF', '#4169E1', '#87CEEB', '#00FFFF', '#20B2AA', '#008B8B', '#1E90FF', '#4682B4', '#5F9EA0','#00BFFF', '#40E0D0', '#00CED1', '#6495ED', '#7B68EE',
              '#6A5ACD', '#483D8B', '#007BA7', '#1CA9C9', '#56A0D3', '#ADD8E6', '#7CB9E8', '#00CCCC', '#1034A6', '#6699CC', '#0073CF', '#0047AB']
need = 4981
for seq_no in tqdm(range(len(bio_annotation_area))):
    if seq_no != need:
        continue
    if len(bio_annotation_area[seq_no]) == 0 or len(no_annotation_area[seq_no]) == 0:
        continue
    if len(bio_annotation_area[seq_no]) > 3 * len(no_annotation_area[seq_no]):
        continue
    p_value = mannwhitneyu(bio_annotation_area[seq_no], no_annotation_area[seq_no], alternative='greater')[1]

    if p_value >= 0.0001:
        continue
   # 找好看的case
    gene_start = 0
    gene_end = test_middle_list[seq_no]['seq_len'] - 1
    functional_regions = {}
    find_list = []
    for annotate in test_middle_list[seq_no]['annotation']:
        find_list.append(annotate['no'])
    k = regulation_annotation.iloc[find_list]
    feature_list = k['Feature type'].tolist()
    functional_regions_name_idx = {'Enhancer': 0, 'TF binding': 0, 'CTCF Binding Site': 0, 'Open chromatin': 0,
       'Promoter': 0, 'SP1': 0, 'Foxo3': 0}
    functional_regions = {}
    for idx in range(len(feature_list)):
        functional_regions_name_idx[feature_list[idx]] += 1
        functional_regions[feature_list[idx] + '_' + str(functional_regions_name_idx[feature_list[idx]])] = (test_middle_list[seq_no]['annotation'][idx]['start_rel'], test_middle_list[seq_no]['annotation'][idx]['end_rel'])
    attention_scores = mean_attention[seq_no]
    input_attention_scores = mean_attention_input[seq_no]
    attention_scores = attention_scores[::10]
    input_attention_scores = input_attention_scores[::10]
    gene_positions = np.arange(1, len(attention_scores) + 1)

    from matplotlib.ticker import MultipleLocator
    plt.rcParams.update({'font.size': 26})

    plt.figure(figsize=(26, 10), dpi=500, facecolor='w')
    plt.rcParams["figure.facecolor"] = "none"
    plt.rcParams["axes.facecolor"] = "none"
    plt.rcParams["legend.facecolor"] = (1, 1, 1, 0)
    plt.plot(gene_positions, attention_scores, marker='.', linestyle='-', color='red', markersize=0.5, linewidth=1, label='Attention Output (single head)')
    plt.plot(gene_positions, input_attention_scores, marker='.', linestyle='--', color='gray', markersize=0.5, linewidth=0.5, alpha=0.7, label='Attention Input')
    plt.xlabel('Relative Gene Position', fontsize=26)
    plt.ylabel('Attention Value', fontsize=26)
    plt.title('Attention Visualization of Gene Segment (middle)', fontsize=26, pad=20)
    color_count = 0
    for region, (start, end) in functional_regions.items():
        plt.fill_between([start // 10, end // 10], [0.15, 0.15], [0.6, 0.6], alpha=0.3, label=region, facecolor=color_list[color_count])
        color_count += 1

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), fancybox=True, shadow=True, ncol=5, fontsize=22)

    plt.xlim(gene_start // 10, gene_end // 10)
    plt.ylim(0.15, 0.6)

    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_edgecolor("black")
        spine.set_linewidth(1)

    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=22)

    ax.yaxis.set_major_locator(MultipleLocator(0.05))

    plt.tight_layout()
    plt.show()
    break

/tmp/ipykernel_2399358/3769327645.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  regulation_annotation = pd.read_table('/data1/personal/baoyh/emo/preprocess_data/with_MA0079.5_0018_0157.tsv', sep='\t')
  0%|          | 0/5787 [00:00<?, ?it/s]

 86%|████████▌ | 4981/5787 [00:03<00:00, 1472.67it/s]


In [15]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

input_file = open('/data1/personal/baoyh/emo/model/pred_results/large/attention_output.npy', 'rb')
attention_output = np.load(input_file)
input_file.close()

input_file = open('/data1/personal/baoyh/emo/model/pred_results/large/attention_input.npy', 'rb')
attention_input = np.load(input_file)
input_file.close()

attention_output = attention_output[:, :, 1, :]

mean_attention = np.abs(attention_output)
mean_attention = np.mean(mean_attention, axis=2)
mean_attention = np.repeat(mean_attention, 100, axis=1)

mean_attention_input = np.abs(attention_input)
mean_attention_input = np.mean(mean_attention_input, axis=2)
mean_attention_input = np.repeat(mean_attention_input, 100, axis=1)


input_file = open('/data1/personal/baoyh/emo/additional/preprocess_data/large/train_large_annotation_MA0079.5_0018_0157_annotation.pkl', 'rb')
test_large_list = pickle.load(input_file)
input_file.close()
bio_annotation_area = []
no_annotation_area = []
for seq_no in tqdm(range(len(mean_attention))):
    bio_annotation_area.append([])
    no_annotation_area.append([])
    if len(test_large_list[seq_no]['annotation']) == 0:
        continue
    for idx in range(test_large_list[seq_no]['seq_len']):
        flag = False
        for bio_idx in range(len(test_large_list[seq_no]['annotation'])):
            if idx >= test_large_list[seq_no]['annotation'][bio_idx]['start_rel'] and idx <= test_large_list[seq_no]['annotation'][bio_idx]['end_rel']:
                bio_annotation_area[seq_no].append(mean_attention[seq_no][idx])
                flag = True
                break
        if not flag:
            no_annotation_area[seq_no].append(mean_attention[seq_no][idx])

100%|██████████| 10058/10058 [1:10:39<00:00,  2.37it/s]


In [4]:
from scipy.stats import mannwhitneyu, ttest_ind
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
from scipy.interpolate import interp1d
regulation_annotation = pd.read_table('/data1/personal/baoyh/emo/preprocess_data/with_MA0079.5_0018_0157.tsv', sep='\t')
color_list = ['#0000FF', '#4169E1', '#87CEEB', '#00FFFF', '#20B2AA', '#008B8B', '#1E90FF', '#4682B4', '#5F9EA0','#00BFFF', '#40E0D0', '#00CED1', '#6495ED', '#7B68EE',
              '#6A5ACD', '#483D8B', '#007BA7', '#1CA9C9', '#56A0D3', '#ADD8E6', '#7CB9E8', '#00CCCC', '#1034A6', '#6699CC', '#0073CF', '#0047AB']
need = 5037
for seq_no in tqdm(range(len(bio_annotation_area))):
    if seq_no != need:
        continue
    if len(bio_annotation_area[seq_no]) == 0 or len(no_annotation_area[seq_no]) == 0:
        continue
    if len(bio_annotation_area[seq_no]) > 3 * len(no_annotation_area[seq_no]):
        continue
    p_value = mannwhitneyu(bio_annotation_area[seq_no], no_annotation_area[seq_no], alternative='greater')[1]

    if p_value >= 0.0001:
        continue
   # 找好看的case
    gene_start = 0
    gene_end = test_large_list[seq_no]['seq_len'] - 1
    functional_regions = {}
    find_list = []
    for annotate in test_large_list[seq_no]['annotation']:
        find_list.append(annotate['no'])
    k = regulation_annotation.iloc[find_list]
    feature_list = k['Feature type'].tolist()
    functional_regions_name_idx = {'Enhancer': 0, 'TF binding': 0, 'CTCF Binding Site': 0, 'Open chromatin': 0,
       'Promoter': 0, 'SP1': 0, 'Foxo3': 0, 'CREB1': 0}
    functional_regions = {}
    for idx in range(len(feature_list)):
        functional_regions_name_idx[feature_list[idx]] += 1
        functional_regions[feature_list[idx] + '_' + str(functional_regions_name_idx[feature_list[idx]])] = (test_large_list[seq_no]['annotation'][idx]['start_rel'], test_large_list[seq_no]['annotation'][idx]['end_rel'])
    attention_scores = mean_attention[seq_no]
    input_attention_scores = mean_attention_input[seq_no]
    attention_scores = attention_scores[::100]
    input_attention_scores = input_attention_scores[::100]
    gene_positions = np.arange(1, len(attention_scores) + 1)

    from matplotlib.ticker import MultipleLocator
    plt.rcParams.update({'font.size': 26})
    # 绘制注意力系数信号峰图
    plt.figure(figsize=(26, 10), dpi=500)
    plt.plot(gene_positions, attention_scores, marker='.', linestyle='-', color='red', markersize=0.5, linewidth=1, label='Attention Output (single head)')
    plt.plot(gene_positions, input_attention_scores, marker='.', linestyle='--', color='gray', markersize=0.5, linewidth=0.5, alpha=0.7, label='Attention Input')
    plt.xlabel('Relative Gene Position', fontsize=26)
    plt.ylabel('Attention Value', fontsize=26)
    plt.title('Attention Visualization of Gene Segment (large)', fontsize=26, pad=20)
    color_count = 0
    for region, (start, end) in functional_regions.items():
        plt.fill_between([start // 100, end // 100], [0.15, 0.15], [0.6, 0.6], alpha=0.3, label=region, facecolor=color_list[color_count])
        color_count += 1

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), fancybox=True, shadow=True, ncol=5, fontsize=22)

    plt.xlim(gene_start // 100, gene_end // 100)
    plt.ylim(0.15, 0.6)

    ax = plt.gca()

    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=22)

    ax.yaxis.set_major_locator(MultipleLocator(0.05))

    plt.tight_layout()
    plt.show()
    break

/tmp/ipykernel_2399358/3576057534.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  regulation_annotation = pd.read_table('/data1/personal/baoyh/emo/preprocess_data/with_MA0079.5_0018_0157.tsv', sep='\t')
100%|██████████| 5787/5787 [00:00<00:00, 1793971.71it/s]
